# Climate Data — Clean CSV and Add Continent/Region


### Data Quality Issue: Continent Aggregates in Country Column
I noticed that the `Country` column sometimes lists entire continents (e.g., *Africa, Asia, Europe*).  
Since most rows represent individual countries, having continents mixed in creates inconsistency.  
I’m going to remove these aggregate rows and instead rely on a new `Continent` feature for geographic grouping.


In [6]:
import pandas as pd
import numpy as np

RAW_PATH = r"C:\Users\Weekseey\Documents\Bellevue Work\Applied Data Science\GlobalTemp_Cleaned.csv"
df_raw = pd.read_csv(RAW_PATH)

print("Raw shape:", df_raw.shape)
display(df_raw.head())

Raw shape: (544811, 8)


,dt,AverageTemperature,AverageTemperatureUncertainty,Country,Year,Month,Decade,Season
0,1743-11-01,4.384,2.294,Åland,1743,11,1740,Fall
1,1744-04-01,1.530,4.680,Åland,1744,4,1740,Spring
2,1744-05-01,6.702,1.789,Åland,1744,5,1740,Spring
3,1744-06-01,11.609,1.577,Åland,1744,6,1740,Summer
4,1744-07-01,15.342,1.410,Åland,1744,7,1740,Summer


In [8]:
# List of continent names that might appear incorrectly in the Country column
CONTINENT_ROWS = ["Africa","Asia","Europe","North America","South America","Oceania","Antarctica"]

# Mask for rows where Country is actually a continent
mask = df_raw["Country"].isin(CONTINENT_ROWS)

# Summary counts
total_rows = df_raw.shape[0]
agg_rows = mask.sum()
unique_aggregates = df_raw.loc[mask, "Country"].nunique()
unique_country_values_total = df_raw["Country"].nunique()
unique_true_countries = df_raw.loc[~mask, "Country"].nunique()

print("Total rows in file:", total_rows)
print("Rows with continent in Country:", agg_rows, f"({agg_rows/total_rows:.2%})")
print("Unique aggregates present:", unique_aggregates)
print("Unique country values total (incl aggregates):", unique_country_values_total)
print("Unique true countries (excl aggregates):", unique_true_countries)
print("Aggregates found:", sorted(df_raw.loc[mask, "Country"].unique().tolist()))

Total rows in file: 544811
Rows with continent in Country: 13790 (2.53%)
Unique aggregates present: 6
Unique country values total (incl aggregates): 242
Unique true countries (excl aggregates): 236
Aggregates found: ['Africa', 'Asia', 'Europe', 'North America', 'Oceania', 'South America']


In [10]:
# Peek at some of the continent rows
display(df_raw.loc[mask, ["dt","Country","Year","Month","Decade","Season","AverageTemperature"]].head(12))

,dt,Country,Year,Month,Decade,Season,AverageTemperature
5251,1850-01-01,Africa,1850,1,1850,Winter,19.764
5252,1850-02-01,Africa,1850,2,1850,Winter,22.393
5253,1850-03-01,Africa,1850,3,1850,Spring,23.672
5254,1850-04-01,Africa,1850,4,1850,Spring,24.197
5255,1850-05-01,Africa,1850,5,1850,Spring,24.302
5256,1850-06-01,Africa,1850,6,1850,Summer,24.980
5257,1850-07-01,Africa,1850,7,1850,Summer,25.089
5258,1850-08-01,Africa,1850,8,1850,Summer,25.254
5259,1850-09-01,Africa,1850,9,1850,Fall,24.919
5260,1850-10-01,Africa,1850,10,1850,Fall,23.831


In [12]:
# Create a new dataframe without those aggregates
df_countries = df_raw[~df_raw["Country"].isin(CONTINENT_ROWS)].copy()

print("Original rows:", df_raw.shape[0])
print("After removing aggregates:", df_countries.shape[0])
print("Unique country values now:", df_countries["Country"].nunique())

Original rows: 544811
After removing aggregates: 531021
Unique country values now: 236


In [22]:
import country_converter as coco

cc = coco.CountryConverter()

unique_countries = (
    df_countries["Country"]
    .dropna()
    .drop_duplicates()
    .sort_values()
    .to_list()
)

mapping = pd.DataFrame({"Country": unique_countries})

mapping["Continent_cc"] = cc.convert(names=mapping["Country"], to="continent")
mapping["UN_Region"]   = cc.convert(names=mapping["Country"], to="UNregion")

print("Unique countries in mapping:", len(mapping))
display(mapping.head(10))

Baker Island not found in regex
Kingman Reef not found in regex
Palmyra Atoll not found in regex
Virgin Islands not found in regex
Baker Island not found in regex
Kingman Reef not found in regex
Palmyra Atoll not found in regex
Virgin Islands not found in regex


Unique countries in mapping: 236


,Country,Continent_cc,UN_Region
0,Afghanistan,Asia,Southern Asia
1,Albania,Europe,Southern Europe
2,Algeria,Africa,Northern Africa
3,American Samoa,Oceania,Polynesia
4,Andorra,Europe,Southern Europe
5,Angola,Africa,Middle Africa
6,Anguilla,America,Caribbean
7,Antigua And Barbuda,America,Caribbean
8,Argentina,America,South America
9,Armenia,Asia,Western Asia


In [24]:
# Step 5 — Normalize "America"/"Americas" and create final 7-continent Continent column

import numpy as np
import pandas as pd

def finalize_continent(cont_cc, un_reg):
    if pd.isna(cont_cc):
        return np.nan

    # Normalize America -> Americas for consistency
    cont = str(cont_cc).strip()
    if cont == "America":
        cont = "Americas"

    # If not Americas, keep as-is (Africa, Asia, Europe, Oceania, Antarctica)
    if cont != "Americas":
        return cont

    # Split the Americas using UN_Region
    r = str(un_reg) if pd.notna(un_reg) else ""

    if "South America" in r:
        return "South America"
    if ("Central America" in r) or ("Caribbean" in r):
        return "North America"
    if "Northern America" in r:
        return "North America"
    if "Latin America" in r:
        return "South America"

    # Fallback if UN_Region is missing
    return "Americas"

mapping["Continent"] = [
    finalize_continent(c, r) for c, r in zip(mapping["Continent_cc"], mapping["UN_Region"])
]

print("Unique Continent_cc:", sorted(mapping["Continent_cc"].dropna().unique().tolist()))
print("Final unique Continent:", sorted(mapping["Continent"].dropna().unique().tolist()))

# Any residual "Americas" or missing values that need manual review
residual = mapping.loc[mapping["Continent"].isna() | (mapping["Continent"] == "Americas"),
                       ["Country","UN_Region","Continent_cc","Continent"]]
print("Residual needing review:", residual.shape[0])
display(residual.head(20))

Unique Continent_cc: ['Africa', 'America', 'Antarctica', 'Asia', 'Europe', 'Oceania', 'not found']
Final unique Continent: ['Africa', 'Antarctica', 'Asia', 'Europe', 'North America', 'Oceania', 'South America', 'not found']
Residual needing review: 0


,Country,UN_Region,Continent_cc,Continent


In [30]:
# Merge mapping back into the country-level data to create your working dataframe

df_cont = df_countries.merge(
    mapping[["Country","Continent","UN_Region","Continent_cc"]],
    on="Country",
    how="left"
)

print("Rows in df_cont:", df_cont.shape[0])
print("Unique countries:", df_cont["Country"].nunique())
print("Unique continents:", df_cont["Continent"].dropna().nunique())
print(sorted(df_cont["Continent"].dropna().unique().tolist()))

# Quick peek
display(df_cont[["dt","Country","Continent","UN_Region","AverageTemperature"]].head(10))

Rows in df_cont: 531021
Unique countries: 236
Unique continents: 8
['Africa', 'Antarctica', 'Asia', 'Europe', 'North America', 'Oceania', 'South America', 'not found']


,dt,Country,Continent,UN_Region,AverageTemperature
0,1743-11-01,Åland,Europe,Northern Europe,4.384
1,1744-04-01,Åland,Europe,Northern Europe,1.530
2,1744-05-01,Åland,Europe,Northern Europe,6.702
3,1744-06-01,Åland,Europe,Northern Europe,11.609
4,1744-07-01,Åland,Europe,Northern Europe,15.342
5,1744-09-01,Åland,Europe,Northern Europe,11.702
6,1744-10-01,Åland,Europe,Northern Europe,5.477
7,1744-11-01,Åland,Europe,Northern Europe,3.407
8,1744-12-01,Åland,Europe,Northern Europe,-2.181
9,1745-01-01,Åland,Europe,Northern Europe,-3.850


In [32]:
# Preview a few examples of countries split into North America and South America

americas_sample = (
    df_cont[df_cont["Continent"].isin(["North America","South America"])]
    [["Country","UN_Region","Continent"]]
    .drop_duplicates()
    .sort_values("Country")
)

print("Total countries in Americas (North + South):", americas_sample.shape[0])
display(americas_sample.head(20))  # first 20 alphabetically

Total countries in Americas (North + South): 53


,Country,UN_Region,Continent
17642,Anguilla,Caribbean,North America
19728,Antigua And Barbuda,Caribbean,North America
21814,Argentina,South America,South America
26222,Aruba,Caribbean,North America
35860,Bahamas,Caribbean,North America
44831,Barbados,Caribbean,North America
53249,Belize,Central America,North America
59574,Bolivia,South America,South America
61474,"Bonaire, Saint Eustatius And Saba",Caribbean,North America
68600,Brazil,South America,South America


In [34]:
# Create a dataframe of all unique countries with their UN_Region and Continent

unique_country_map = (
    df_cont[["Country","UN_Region","Continent"]]
    .drop_duplicates()
    .sort_values("Country")
    .reset_index(drop=True)
)

print("Unique countries in mapping:", unique_country_map.shape[0])
display(unique_country_map.head(20))

# Optional: save to CSV for manual review
unique_country_map.to_csv("Unique_Country_UNRegion_Continent.csv", index=False)
print("Saved: Unique_Country_UNRegion_Continent.csv")

Unique countries in mapping: 236


,Country,UN_Region,Continent
0,Afghanistan,Southern Asia,Asia
1,Albania,Southern Europe,Europe
2,Algeria,Northern Africa,Africa
3,American Samoa,Polynesia,Oceania
4,Andorra,Southern Europe,Europe
5,Angola,Middle Africa,Africa
6,Anguilla,Caribbean,North America
7,Antigua And Barbuda,Caribbean,North America
8,Argentina,South America,South America
9,Armenia,Western Asia,Asia


Saved: Unique_Country_UNRegion_Continent.csv


In [36]:
# List countries where UN_Region or Continent is 'not found'

not_found = (
    unique_country_map[
        (unique_country_map["UN_Region"].astype(str).str.lower() == "not found") |
        (unique_country_map["Continent"].astype(str).str.lower() == "not found")
    ]
    .sort_values("Country")
    .reset_index(drop=True)
)

print("Countries with 'not found' in UN_Region or Continent:", not_found.shape[0])
display(not_found)

Countries with 'not found' in UN_Region or Continent: 4


,Country,UN_Region,Continent
0,Baker Island,not found,not found
1,Kingman Reef,not found,not found
2,Palmyra Atoll,not found,not found
3,Virgin Islands,not found,not found


In [40]:
# Manually patch the 4 "not found" countries into North America / Northern America

manual_fixes = {
    "Baker Island":   ("North America", "Northern America"),
    "Kingman Reef":   ("North America", "Northern America"),
    "Palmyra Atoll":  ("North America", "Northern America"),
    "Virgin Islands": ("North America", "Caribbean"),
}

# --- Update unique_country_map ---
mask_ucm = unique_country_map["Country"].isin(manual_fixes)
unique_country_map.loc[mask_ucm, "Continent"] = unique_country_map.loc[mask_ucm, "Country"].map(
    lambda c: manual_fixes[c][0]
)
unique_country_map.loc[mask_ucm, "UN_Region"] = unique_country_map.loc[mask_ucm, "Country"].map(
    lambda c: manual_fixes[c][1]
)

# --- Update df_cont as well ---
mask_df = df_cont["Country"].isin(manual_fixes)
df_cont.loc[mask_df, "Continent"] = df_cont.loc[mask_df, "Country"].map(lambda c: manual_fixes[c][0])
df_cont.loc[mask_df, "UN_Region"] = df_cont.loc[mask_df, "Country"].map(lambda c: manual_fixes[c][1])

# Verify fixes
print("Remaining 'not found' rows in unique_country_map:",
      unique_country_map[
          (unique_country_map["UN_Region"].astype(str).str.lower()=="not found") |
          (unique_country_map["Continent"].astype(str).str.lower()=="not found")
      ].shape[0])

display(unique_country_map[unique_country_map["Country"].isin(manual_fixes.keys())])

Remaining 'not found' rows in unique_country_map: 0


,Country,UN_Region,Continent
16,Baker Island,Northern America,North America
113,Kingman Reef,Northern America,North America
162,Palmyra Atoll,Northern America,North America
230,Virgin Islands,Caribbean,North America


In [46]:
df_cont = df_cont.drop(columns=["Continent_cc"])

### Data Quality Issue: Duplicate Country Names with Suffixes
Some countries appear more than once, such as *Denmark* and *Denmark (Europe)*, or *France* and *France (Europe)*.  
These duplicates even share the same dates but have different temperature values, which could lead to double-counting.  
I’m going to drop the suffix versions and keep only the clean country names.


In [55]:
# Drop duplicate country names with "(Europe)" suffix

# Identify rows with suffix
suffix_mask = df_cont["Country"].str.contains(r"\(Europe\)", na=False)

print("Rows with (Europe) suffix:", suffix_mask.sum())
print("Unique affected countries:", df_cont.loc[suffix_mask, "Country"].nunique())
print("Examples:", df_cont.loc[suffix_mask, "Country"].unique()[:10])

Rows with (Europe) suffix: 12664
Unique affected countries: 4
Examples: ['Denmark (Europe)' 'France (Europe)' 'Netherlands (Europe)'
 'United Kingdom (Europe)']


In [57]:
# Drop those rows
df_cont = df_cont[~suffix_mask].copy()

print("Remaining unique countries:", df_cont["Country"].nunique())

Remaining unique countries: 232


### Data Quality Issue: Ambiguous Country Names
I also found a few cases where country names are ambiguous or inconsistent:
- *Congo* vs *Congo (Democratic Republic Of The)*  
- *Virgin Islands* vs *British Virgin Islands*  

To keep the data clear, I’m going to standardize these so each country is represented uniquely:
- Congo → Republic of the Congo  
- Congo (Democratic Republic Of The) → Democratic Republic of the Congo  
- Virgin Islands → United States Virgin Islands


In [59]:
# Standardize ambiguous country names

# Define mapping for ambiguous names
country_replacements = {
    "Congo": "Republic of the Congo",
    "Congo (Democratic Republic Of The)": "Democratic Republic of the Congo",
    "Virgin Islands": "United States Virgin Islands"
}

# Apply replacements
df_cont["Country"] = df_cont["Country"].replace(country_replacements)

# Verify the fixes
for old, new in country_replacements.items():
    print(f"{old} → {new}:",
          df_cont[df_cont["Country"] == new]["Country"].unique())

Congo → Republic of the Congo: ['Republic of the Congo']
Congo (Democratic Republic Of The) → Democratic Republic of the Congo: ['Democratic Republic of the Congo']
Virgin Islands → United States Virgin Islands: ['United States Virgin Islands']


### Data Quality Note: Datetime Column
When opening the cleaned CSV in Excel, some dates may appear in different formats (e.g., `1823-01-01` vs `8/1/2013`). This happens because Excel automatically reformats recognized dates and leaves earlier years (before 1900) as text.  

For analysis purposes inside Python, the dates are stored consistently as `datetime64[ns]` and are reliable for grouping, resampling, and time-series modeling.


In [61]:
# Check current data type for 'dt' column

print("\nColumn 'dt' dtype:", df_cont["dt"].dtype)


Column 'dt' dtype: object


In [67]:
# Standardize inconsistent date formats in `dt`

# Convert all date strings to datetime objects
df_cont["dt"] = pd.to_datetime(df_cont["dt"], errors="coerce", infer_datetime_format=True)

# Quick QA
print("Column 'dt' dtype after conversion:", df_cont["dt"].dtype)
print("Number of missing/unparseable dates:", df_cont["dt"].isna().sum())
print("Date range:", df_cont["dt"].min(), "→", df_cont["dt"].max())

Column 'dt' dtype after conversion: datetime64[ns]
Number of missing/unparseable dates: 0
Date range: 1743-11-01 00:00:00 → 2013-09-01 00:00:00


C:\Users\Weekseey\AppData\Local\Temp\ipykernel_3616\68610021.py:4: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_cont["dt"] = pd.to_datetime(df_cont["dt"], errors="coerce", infer_datetime_format=True)


In [69]:
# Keep datetime64[ns], just format for display
df_display = df_cont.copy()
df_display["dt"] = df_display["dt"].dt.strftime("%Y-%m-%d")

display(df_display.head())

,dt,AverageTemperature,AverageTemperatureUncertainty,Country,Year,Month,Decade,Season,Continent,UN_Region
0,1743-11-01,4.384,2.294,Åland,1743,11,1740,Fall,Europe,Northern Europe
1,1744-04-01,1.530,4.680,Åland,1744,4,1740,Spring,Europe,Northern Europe
2,1744-05-01,6.702,1.789,Åland,1744,5,1740,Spring,Europe,Northern Europe
3,1744-06-01,11.609,1.577,Åland,1744,6,1740,Summer,Europe,Northern Europe
4,1744-07-01,15.342,1.410,Åland,1744,7,1740,Summer,Europe,Northern Europe


In [71]:
print("\nColumn 'dt' dtype:", df_cont["dt"].dtype)


Column 'dt' dtype: datetime64[ns]


### Rebuilding Unique Country Map
Now that the country names have been cleaned and standardized, I’m going to rebuild the unique country-to-continent/region mapping.  
This ensures the mapping reflects only valid country names without duplicates or ambiguous entries.


In [75]:
# Rebuild unique_country_map after cleaning df_cont

unique_country_map = (
    df_cont[["Country","UN_Region","Continent"]]
    .drop_duplicates()
    .sort_values("Country")
    .reset_index(drop=True)
)

print("Updated unique_country_map shape:", unique_country_map.shape)
display(unique_country_map.head(20))

Updated unique_country_map shape: (232, 3)


,Country,UN_Region,Continent
0,Afghanistan,Southern Asia,Asia
1,Albania,Southern Europe,Europe
2,Algeria,Northern Africa,Africa
3,American Samoa,Polynesia,Oceania
4,Andorra,Southern Europe,Europe
5,Angola,Middle Africa,Africa
6,Anguilla,Caribbean,North America
7,Antigua And Barbuda,Caribbean,North America
8,Argentina,South America,South America
9,Armenia,Western Asia,Asia


In [81]:
# Save the cleaned dataset and the unique country→region mapping (UTF-8 with BOM for Excel)

# Cleaned analysis dataset with Continent/UN_Region (no continent-aggregate rows)
CLEAN_PATH = "GlobalTemp_Cleaned_CountryContinent.csv"
df_cont.to_csv(CLEAN_PATH, index=False, encoding="utf-8-sig")
print("Saved cleaned dataset:", CLEAN_PATH, "| Rows:", df_cont.shape[0])

# Unique Country → UN_Region/Continent table for manual review or sharing
MAP_PATH = "Unique_Country_UNRegion_Continent.csv"
unique_country_map.to_csv(MAP_PATH, index=False, encoding="utf-8-sig")
print("Saved mapping table:", MAP_PATH, "| Rows:", unique_country_map.shape[0])

Saved cleaned dataset: GlobalTemp_Cleaned_CountryContinent.csv | Rows: 518357
Saved mapping table: Unique_Country_UNRegion_Continent.csv | Rows: 232
